<a href="https://colab.research.google.com/github/Om-Shree-0709/Pytorch/blob/main/PCA_%26_its_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PCA : principal component analysis [Pre-Processing Techniques] \
D:{x,y} : given dataset (supervised dataset)
X belongs to R raised to power d [features] : dimensionality->x ke features

D conveted to D' : {x,y} X belongs to R raised to power d' \
( where d'< d always )

TO REDUCE THE DIMENSIONALITY OF FEATURES \
because : faster computation and training

AXIS ROTATION:
1. Find covarience Matrix
2. FInd Eigen Values
3. Find resp Eigen Vectors
4. Sort in  Decending order and pick the top 3

:In Starting lets assume we had dimensionality=10 \
now we r left with only 3 dimensions

OBJECTIVE FXN:
1. SPREAD Maximization
2. Distance Minimization
Before implementing PCA : we need to standardize the data \
mean = 0 \
variance = 1 \
( for each column )

We have to find u1(unit vector) in such a way that , Variance of the Projected print is maximized

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
